In [30]:
from sklearn.model_selection import train_test_split

In [31]:
import os
import pandas as pd


In [32]:
import fasttext

In [33]:

with open("/14td1/varun-18869/model/create_finetune/finetune/Samanantar.en-mr.en", "r") as f1, open("/14td1/varun-18869/model/create_finetune/finetune/Samanantar.en-mr.mr", "r") as f2:
    src_lines = f1.readlines()
    tgt_lines = f2.readlines()

In [34]:
df = pd.DataFrame({"source" : src_lines, "target": tgt_lines})

In [35]:
df.head()

,source,target
0,"Today, ASEAN and India enjoy multi-faceted coo...","आज आसियान आणि भारत, आसियानचे राजनीतिक- सुरक्षा..."
1,"For us in India, ASEANs continuance in this ro...",आसियान राष्ट्र समुहातील देशांशी भारताचे उत्तम ...
2,We need to put faith in his reminders. \n,[ ७ पानांवरील चित्र] \n
3,Presently elections are being held in Gujarat. \n,"दरम्यान, गुजरात निवडणूकीचा प्रचार आता शिगेला प..."
4,This will aim to pool research and technologic...,यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला आणि त...


In [36]:
# Checking for NULL or Empty Values
faulty_indices = df[df.isnull().any(axis=1)].index
empty_indices = df[df.isna().any(axis=1)].index
print(faulty_indices)
print(empty_indices)

cleaned_data = df.drop(index=faulty_indices)
cleaned_data = df.drop(index=empty_indices)

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


In [37]:
# Removing all \n's from all the sentences.
# Remove newline characters and converting everything to lowercase for source sentences to avoid increased vocab.
df['source'] = df['source'].str.replace('\n', '').str.lower()
df['target'] = df['target'].str.replace('\n', '')

In [38]:
# Removing those lines which contains words not part of their respective langauge.
# Load pre-trained FastText language detection model
model = fasttext.load_model('/14td1/varun-18869/model/models/fasttext_model/lid.176.bin')

# Function to detect language
def detect_language(text):
    prediction = model.predict(text)
    return prediction[0][0].replace('__label__', '')

# Detect languages for source and target sentences
df['source_language'] = df['source'].apply(detect_language) #Function called here
df['target_language'] = df['target'].apply(detect_language) #Function called here

In [39]:
df.head()

,source,target,source_language,target_language
0,"today, asean and india enjoy multi-faceted coo...","आज आसियान आणि भारत, आसियानचे राजनीतिक- सुरक्षा...",en,mr
1,"for us in india, aseans continuance in this ro...",आसियान राष्ट्र समुहातील देशांशी भारताचे उत्तम ...,en,mr
2,we need to put faith in his reminders.,[ ७ पानांवरील चित्र],en,mr
3,presently elections are being held in gujarat.,"दरम्यान, गुजरात निवडणूकीचा प्रचार आता शिगेला प...",en,mr
4,this will aim to pool research and technologic...,यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला आणि त...,en,mr


In [40]:
# Remove rows where the detected language does not match the majority language
#In the case we dont know source and target languages
# Identify the majority language for source and target columns
"""
import Counter
source_language_counts = Counter(df['source_language'])
target_language_counts = Counter(df['target_language'])

majority_source_language = source_language_counts.most_common(1)[0][0]
majority_target_language = target_language_counts.most_common(1)[0][0]
"""
source_language = "en"
target_language = "mr"

cleaned_data = df[
    (df['source_language'] == source_language) &
    (df['target_language'] == target_language)
]


In [41]:
cleaned_data.__len__()

3575532

In [42]:
# Remove duplicate source sentences along with their corresponding target sentences
#Since we are not using de-duplication here due to small dataset.
"""
cleaned_data = cleaned_data.drop_duplicates(subset='source', keep=False)
"""
cleaned_data.__len__()

3575532

In [43]:
#Removing those lines having target sentence characters X2 or /2 means less or more than 2 times to that of source.

# Calculate character counts for source and target sentences
cleaned_data['source_char_count'] = cleaned_data['source'].str.len()
cleaned_data['target_char_count'] = cleaned_data['target'].str.len()

# Filter out rows where the target character count is either double or half of the source character count
cleaned_data = cleaned_data[
    (cleaned_data['target_char_count'] < 2 * cleaned_data['source_char_count']) |
    (cleaned_data['target_char_count'] > cleaned_data['source_char_count'] / 2)
]

df  = cleaned_data

/tmp/ipykernel_3598185/2428176987.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['source_char_count'] = cleaned_data['source'].str.len()
/tmp/ipykernel_3598185/2428176987.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['target_char_count'] = cleaned_data['target'].str.len()


In [44]:
df.head()

,source,target,source_language,target_language,source_char_count,target_char_count
0,"today, asean and india enjoy multi-faceted coo...","आज आसियान आणि भारत, आसियानचे राजनीतिक- सुरक्षा...",en,mr,301,401
1,"for us in india, aseans continuance in this ro...",आसियान राष्ट्र समुहातील देशांशी भारताचे उत्तम ...,en,mr,158,175
2,we need to put faith in his reminders.,[ ७ पानांवरील चित्र],en,mr,39,21
3,presently elections are being held in gujarat.,"दरम्यान, गुजरात निवडणूकीचा प्रचार आता शिगेला प...",en,mr,47,58
4,this will aim to pool research and technologic...,यामुळे सौर ऊर्जा क्षेत्रातल्या संशोधनाला आणि त...,en,mr,183,169


In [45]:
# Splitting would the last activity to be perfomed before that dataset cleaning is done

#Drop the language columns and character count columns as they are no longer needed
df = df.drop(columns=['source_language', 'target_language', 'source_char_count', 'target_char_count'])

# Splitting into train and test/validation
train_data, test_valid_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting test/validation into test and validation
test_data, valid_data = train_test_split(test_valid_data, test_size=0.5, random_state=42)

In [46]:
# Creating files for train test valid for each source and target
train_source_path = "/14td1/varun-18869/model/create_finetune/finetune/train_source.txt"
train_target_path = "/14td1/varun-18869/model/create_finetune/finetune/train_target.txt"
test_source_path = "/14td1/varun-18869/model/create_finetune/finetune/test_source.txt"
test_target_path = "/14td1/varun-18869/model/create_finetune/finetune/test_target.txt"
valid_source_path = "/14td1/varun-18869/model/create_finetune/finetune/valid_source.txt"
valid_target_path = "/14td1/varun-18869/model/create_finetune/finetune/valid_target.txt"

# Save train, test, and validation datasets to text files
train_data[['source']].to_csv(train_source_path, index=False, header=False)
train_data[['target']].to_csv(train_target_path, index=False, header=False)

test_data[['source']].to_csv(test_source_path, index=False, header=False)
test_data[['target']].to_csv(test_target_path, index=False, header=False)

valid_data[['source']].to_csv(valid_source_path, index=False, header=False)
valid_data[['target']].to_csv(valid_target_path, index=False, header=False)

In [47]:
#Then comes the tokenization step.
"""
spm_train --input=total.txt --model_prefix=your_model --vocab_size=32000 --character_coverage=1.0 \--model_type=bpe --input_sentence_size=300000000  --shuffle_input_sentence=true --user_defined_symbols="__email0__","__email1__","__email2__","__email3__","__email4__","__url0__","__url1__","__url2__"
"""

'\nspm_train --input=total.txt --model_prefix=your_model --vocab_size=32000 --character_coverage=1.0 \\--model_type=bpe --input_sentence_size=300000000  --shuffle_input_sentence=true --user_defined_symbols="__email0__","__email1__","__email2__","__email3__","__email4__","__url0__","__url1__","__url2__"\n'

In [48]:
# Encoding the dataset into pieces -> i.e tokenizing the dataset

# spm_encode --model=your_model.model --output_format=piece < train_file.$src > spm_encoded_train.$src &
# spm_encode --model=your_model.model --output_format=piece < train_file.$trg > spm_encoded_train.$trg &

In [49]:
#Then Comes Preprocessing step for optimizations.
"""
num_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`
    fairseq-preprocess --source-lang en --target-lang hi \
             --trainpref train --validpref valid --testpref test \
             --destdir data-bin --nwordssrc 32000 --nwordstgt 32000 --workers $num_workers \
             --log-interval 1 \
             --log-format tqdm \
"""

'\nnum_workers=`python -c "import multiprocessing; print(multiprocessing.cpu_count())"`\n    fairseq-preprocess --source-lang en --target-lang hi              --trainpref train --validpref valid --testpref test              --destdir data-bin --nwordssrc 32000 --nwordstgt 32000 --workers $num_workers              --log-interval 1              --log-format tqdm '

In [50]:
# Then Comes the Training Step
"""
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 fairseq-train  data-bin/  --arch transformer --save-dir checkpoints_en_hi/  \
         --log-interval 2 \
         --log-format tqdm \
         --source-lang=en --target-lang=hi \
         --encoder-embed-dim 512  --encoder-ffn-embed-dim 1024 --encoder-attention-heads 8 \
         --decoder-embed-dim 512  --decoder-ffn-embed-dim 1024 --decoder-attention-heads 8 \
         --optimizer adam --adam-betas '(0.9, 0.98)'\
         --clip-norm 1.0 --lr 0.0005 --lr-scheduler inverse_sqrt  --warmup-updates 4000 \
         --warmup-init-lr 1e-07 --dropout 0.3  --weight-decay 0.001   --criterion label_smoothed_cross_entropy  \
         --label-smoothing 0.1 --max-tokens 1024 --fp16 --update-freq 16 --max-epoch 10 --eval-bleu \
         --eval-bleu-remove-bpe \
         --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
         --tensorboard-logdir data_en_hi/logs \
         --wandb-project transformer_en_hi \
         --seed 1234 --find-unused-parameters --skip-invalid-size-inputs-valid-test
"""

"\nCUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 fairseq-train  data-bin/  --arch transformer --save-dir checkpoints_en_hi/           --log-interval 2          --log-format tqdm          --source-lang=en --target-lang=hi          --encoder-embed-dim 512  --encoder-ffn-embed-dim 1024 --encoder-attention-heads 8          --decoder-embed-dim 512  --decoder-ffn-embed-dim 1024 --decoder-attention-heads 8          --optimizer adam --adam-betas '(0.9, 0.98)'         --clip-norm 1.0 --lr 0.0005 --lr-scheduler inverse_sqrt  --warmup-updates 4000          --warmup-init-lr 1e-07 --dropout 0.3  --weight-decay 0.001   --criterion label_smoothed_cross_entropy           --label-smoothing 0.1 --max-tokens 1024 --fp16 --update-freq 16 --max-epoch 10 --eval-bleu          --eval-bleu-remove-bpe          --best-checkpoint-metric bleu --maximize-best-checkpoint-metric          --tensorboard-logdir data_en_hi/logs          --wandb-project transformer_en_hi          --seed 1234 --find-unused-parameters --skip-invalid

In [51]:
# Then comes inference or generation step.
"""
CUDA_VISIBLE_DEVICES="4,5,6" fairseq-interactive data-bin \
    --path ./model_output/checkpoint_best.pt \
    --task translation \
    --source-lang en \
    --target-lang hi \
    --beam 5 \
    --remove-bpe=sentencepiece \
    --buffer-size 128 \
    --batch-size 128 \
    --log-format tqdm \
    --log-interval 60 \
    --input input.txt \
    > output.txt
"""

'\n\n'

In [52]:
# Taking inference from above teacher model, Now to create student model from it.

In [53]:
# Tokenizing the student model

In [54]:
# Training the student model

In [55]:
# Fairseq-generate inferencing is done on student model

In [56]:
# Now, for fast and optimized inference converting the model to ctranslate2 format.
"""ct2-fairseq-converter --model_path model_output/checkpoint_best.pt --data_dir ./data-bin/ --source_lang en --target_lang mr --output_dir ct2_model --quantization int8 --force"""

In [57]:
#Now getting inference from ctranslate2 model.

In [58]:
# Now fine tuning the above model for similar language to that of hindi